In [1]:

import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [2]:
df = pd.read_csv('/Users/joaosequeira/udacity_joao/atp_analysis/atp_tennis.csv')


In [3]:
df.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
0,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mayer F.,Giraldo S.,Mayer F.,28,57,1215,778,1.36,3.00,6-4 6-4
1,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,Nieminen J.,Nieminen J.,35,41,1075,927,2.20,1.61,3-6 6-2 1-6
2,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Nishikori K.,Matosevic M.,Nishikori K.,19,49,1830,845,1.25,3.75,7-5 6-2
3,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mitchell B.,Baghdatis M.,Baghdatis M.,326,36,137,1070,9.00,1.07,4-6 4-6
4,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,Klizan M.,Istomin D.,43,30,897,1175,1.90,1.80,6-1 6-2


In [4]:


def winner_metrics(col_x, col_y, df, winner_col='Winner', player1_col='Player_1', player2_col='Player_2'):
    """
    Função que extrai valores de colunas de um DataFrame, de acordo com condições de outras colunas.

    Argumentos:
    df -- DataFrame Pandas a ser utilizado.
    winner_col -- Nome da coluna que será comparada com "player1_col" e "player2_col".
    player1_col -- Nome da coluna que será comparada com "winner_col".
    player2_col -- Nome da coluna que será comparada com "winner_col".
    col_x -- Nome da coluna de onde será extraído o valor caso "player1_col=winner_col".
    col_y -- Nome da coluna de onde será extraído o valor caso "player2_col=winner_col".

    Retorna:
    Um DataFrame com as colunas "winner" e "odd".
    """

    # Inicializa dicionários para armazenar os valores extraídos
    result_dict = {"winner": [], "metric": []}

    # Itera sobre as linhas do DataFrame
    for index, row in df.iterrows():

        # Compara os valores das colunas "winner_col" e "player1_col"
        if row[player1_col] == row[winner_col]:
            result_dict["winner"].append(row[player1_col])
            result_dict["metric"].append(row[col_x])

        # Compara os valores das colunas "winner_col" e "player2_col"
        elif row[player2_col] == row[winner_col]:
            result_dict["winner"].append(row[player2_col])
            result_dict["metric"].append(row[col_y])

    # Retorna um DataFrame com as colunas "winner" e "odd"
    return pd.DataFrame(result_dict)


In [5]:
# Define uma função que retorna o vencedor de uma partida
def get_winner_rank(df):
    if df['Winner'] == df['Player_1']:
        return df['Rank_1']
    else:
        return df['Rank_2']
    
# Define uma função que retorna o perdedor de uma partida
def get_loser_rank(df):
    if df['Winner'] != df['Player_1']:
        return df['Rank_1']
    else:
        return df['Rank_2']
    
# Aplica a função ao DataFrame para criar a coluna "Loser"
df['Rank_loser'] = df.apply(get_loser_rank, axis=1)
df['Rank_winner'] = df.apply(get_winner_rank, axis=1)

# Criando df com os ganhadores tendo rank pior
df_loser_bigger_rank = df[df['Rank_winner'] > df['Rank_loser']]

df_winner_bigger_rank = df[df['Rank_winner'] < df['Rank_loser']]

count_loser_bigger_rank = len(df_loser_bigger_rank)/len(df)
print("Número de eventos onde o rank do vencedor é maior do que o do perdedor: ", count_loser_bigger_rank)



Número de eventos onde o rank do vencedor é maior do que o do perdedor:  0.351391846068922


In [6]:

print("Porcentagem de partidas ganhas por quadra quando o vencedor tem menor ranking: ", df_loser_bigger_rank['Court'].value_counts()/len(df_loser_bigger_rank))

print("Porcentagem de partidas ganhas por quadra quando o vencedor tem maior ranking: ", df_winner_bigger_rank['Court'].value_counts()/len(df_winner_bigger_rank))

print("Porcentagem de partidas ganhas por quadra no total: ", df['Court'].value_counts()/len(df))

Porcentagem de partidas ganhas por quadra quando o vencedor tem menor ranking:  Court
Outdoor    0.815866
Indoor     0.184134
Name: count, dtype: float64
Porcentagem de partidas ganhas por quadra quando o vencedor tem maior ranking:  Court
Outdoor    0.822918
Indoor     0.177082
Name: count, dtype: float64
Porcentagem de partidas ganhas por quadra no total:  Court
Outdoor    0.82044
Indoor     0.17956
Name: count, dtype: float64


In [7]:

print("Porcentagem de partidas ganhas por Surface quando o vencedor tem menor ranking: ", df_loser_bigger_rank['Surface'].value_counts()/len(df_loser_bigger_rank))

print("Porcentagem de partidas ganhas por Surface quando o vencedor tem maior ranking: ", df_winner_bigger_rank['Surface'].value_counts()/len(df_winner_bigger_rank))

print("Porcentagem de partidas ganhas por Surface no total: ", df['Surface'].value_counts()/len(df))

Porcentagem de partidas ganhas por Surface quando o vencedor tem menor ranking:  Surface
Hard     0.576189
Clay     0.312276
Grass    0.111535
Name: count, dtype: float64
Porcentagem de partidas ganhas por Surface quando o vencedor tem maior ranking:  Surface
Hard     0.589483
Clay     0.299757
Grass    0.110760
Name: count, dtype: float64
Porcentagem de partidas ganhas por Surface no total:  Surface
Hard     0.584812
Clay     0.304156
Grass    0.111032
Name: count, dtype: float64


In [8]:

print("Porcentagem de partidas ganhas por round quando o vencedor tem menor ranking: ", df_loser_bigger_rank['Round'].value_counts()/len(df_loser_bigger_rank))

print("Porcentagem de partidas ganhas por round quando o vencedor tem maior ranking: ", df_winner_bigger_rank['Round'].value_counts()/len(df_winner_bigger_rank))

print("Porcentagem de partidas ganhas por round no total: ", df['Round'].value_counts()/len(df))

Porcentagem de partidas ganhas por round quando o vencedor tem menor ranking:  Round
1st Round        0.475875
2nd Round        0.265821
Quarterfinals    0.101324
3rd Round        0.058685
Semifinals       0.053411
The Final        0.025920
4th Round        0.014475
Round Robin      0.004488
Name: count, dtype: float64
Porcentagem de partidas ganhas por round quando o vencedor tem maior ranking:  Round
1st Round        0.429909
2nd Round        0.296839
Quarterfinals    0.098359
3rd Round        0.075015
Semifinals       0.048815
The Final        0.024924
4th Round        0.021277
Round Robin      0.004863
Name: count, dtype: float64
Porcentagem de partidas ganhas por round no total:  Round
1st Round        0.446061
2nd Round        0.285940
Quarterfinals    0.099401
3rd Round        0.069277
Semifinals       0.050430
The Final        0.025274
4th Round        0.018887
Round Robin      0.004731
Name: count, dtype: float64


In [9]:

print("Porcentagem de partidas ganhas por series quando o vencedor tem menor ranking: ", df_loser_bigger_rank['Series'].value_counts()/len(df_loser_bigger_rank))

print("Porcentagem de partidas ganhas por series quando o vencedor tem maior ranking: ", df_winner_bigger_rank['Series'].value_counts()/len(df_winner_bigger_rank))

print("Porcentagem de partidas ganhas por series no total: ", df['Series'].value_counts()/len(df))

Porcentagem de partidas ganhas por series quando o vencedor tem menor ranking:  Series
ATP250          0.466001
Masters 1000    0.205790
ATP500          0.163824
Grand Slam      0.158550
Masters Cup     0.005835
Name: count, dtype: float64
Porcentagem de partidas ganhas por series quando o vencedor tem maior ranking:  Series
ATP250          0.403769
Grand Slam      0.221763
Masters 1000    0.212401
ATP500          0.156109
Masters Cup     0.005957
Name: count, dtype: float64
Porcentagem de partidas ganhas por series no total:  Series
ATP250          0.425637
Masters 1000    0.210078
Grand Slam      0.199551
ATP500          0.158820
Masters Cup     0.005914
Name: count, dtype: float64


In [10]:
'''
This code receives a DataFrame named 'df' and performs the following operations:

Selects 11 columns from 'df' and assigns them to a new DataFrame named 'prep_treated_df'.
Converts the 'Date' column in 'prep_treated_df' to datetime format.
Defines a function named 'get_winner_loser_values' that assigns a value of 1 to the winner and -1 to the loser in each row of 'prep_treated_df', and applies this function to each row of the DataFrame, assigning the results to new columns named 'Player_1_value' and 'Player_2_value'.
Creates a new DataFrame named 'df_momentum' that selects 5 columns from 'prep_treated_df' and renames them to 'Date', 'Player_1', 'Player_2', 'Value_1', and 'Value_2', respectively.
Defines a function named 'stack_columns_and_running_sum' that concatenates the 'Date', 'Player_1', and 'Value_1' columns of 'df_momentum' with the 'Date', 'Player_2', and 'Value_2' columns of 'df_momentum' (renamed to 'Player' and 'Values', respectively), resulting in a new DataFrame named 'df_stacked'. This function also calculates the cumulative sum of the 'Values' column partitioned by 'Player' and ordered by 'Date', and assigns the result to a new column named 'Cumulative_Sum' in 'df_stacked'.
Applies the 'stack_columns_and_running_sum' function to 'df_momentum', assigning the result to 'df_momentum'.
Then, merges trated_df with momentum dataframe.
'''

prep_treated_df = df[['Date','Series','Court','Surface','Round','Best of','Player_1','Rank_1','Player_2','Rank_2','Rank_winner','Rank_loser']]

prep_treated_df['Date'] = pd.to_datetime(prep_treated_df['Date'])

#Funcao para definir valores aos vencedores e perdedores
def get_winner_loser_values(row):
    if row['Rank_winner'] == row['Rank_1']:
        return pd.Series({'Winner_Value': 1, 'Loser_Value': -1})
    else:
        return pd.Series({'Winner_Value': -1, 'Loser_Value': 1})

prep_treated_df[['Value_1', 'Value_2']] = prep_treated_df.apply(get_winner_loser_values, axis=1)


df_momentum = prep_treated_df[['Date','Player_1','Player_2','Value_1','Value_2']]

#criando o dataframe com o momento
def stack_columns_and_running_sum(df):
    # Cria um novo DataFrame com as colunas empilhadas
    df_stacked = pd.concat([df[['Date', 'Player_1', 'Value_1']].rename(columns={'Player_1': 'Player', 'Value_1': 'Values'}), 
                            df[['Date', 'Player_2', 'Value_2']].rename(columns={'Player_2': 'Player', 'Value_2': 'Values'})])
    
    # Calcula a soma cumulativa da coluna "Values" particionando por "Player" e ordenando por "Date"
    df_stacked = df_stacked.sort_values(['Player', 'Date'])
    df_stacked['Cumulative_Sum'] = df_stacked.groupby('Player')['Values'].cumsum().reset_index(drop=True) 
    df_stacked['3_matches_sum'] = df_stacked.groupby('Player')['Values'].rolling(window=3).sum().reset_index(drop=True) 
    df_stacked['5_matches_sum'] = df_stacked.groupby('Player')['Values'].rolling(window=5).sum().reset_index(drop=True) 
    df_stacked['10_matches_sum'] = df_stacked.groupby('Player')['Values'].rolling(window=10).sum().reset_index(drop=True) 
    df_stacked['15_matches_sum'] = df_stacked.groupby('Player')['Values'].rolling(window=15).sum().reset_index(drop=True) 
    df_stacked['20_matches_sum'] = df_stacked.groupby('Player')['Values'].rolling(window=20).sum().reset_index(drop=True) 


    return df_stacked[['Date', 'Player','Cumulative_Sum','3_matches_sum','5_matches_sum','10_matches_sum','15_matches_sum','20_matches_sum']]


df_momentum = stack_columns_and_running_sum(df_momentum)

treated_df = pd.merge(prep_treated_df, df_momentum, left_on=['Date', 'Player_1'], right_on=['Date', 'Player'])
treated_df = pd.merge(treated_df, df_momentum, left_on=['Date', 'Player_2'], right_on=['Date', 'Player']).rename(columns={'Cumulative_Sum_x': 'Momentum_1', 'Cumulative_Sum_y': 'Momentum_2'}).drop(columns=['Player_x','Player_y'])

/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_2602/2201121846.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_treated_df['Date'] = pd.to_datetime(prep_treated_df['Date'])
/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_2602/2201121846.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_treated_df[['Value_1', 'Value_2']] = prep_treated_df.apply(get_winner_loser_values, axis=1)
/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_2602/2201121846.py:24: SettingWith

In [12]:
treated_df


,Date,Series,Court,Surface,Round,Best of,Player_1,Rank_1,Player_2,Rank_2,...,5_matches_sum_x,10_matches_sum_x,15_matches_sum_x,20_matches_sum_x,Momentum_2,3_matches_sum_y,5_matches_sum_y,10_matches_sum_y,15_matches_sum_y,20_matches_sum_y
0,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mayer F.,28,Giraldo S.,57,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN
1,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,35,Nieminen J.,41,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Nishikori K.,19,Matosevic M.,49,...,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
3,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mitchell B.,326,Baghdatis M.,36,...,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,NaN,NaN
4,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,43,Klizan M.,30,...,1.0,NaN,NaN,NaN,1,1.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26408,2023-03-30,Masters 1000,Outdoor,Hard,Quarterfinals,3,Cerundolo F.,31,Khachanov K.,16,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
26409,2023-03-31,Masters 1000,Outdoor,Hard,Quarterfinals,3,Alcaraz C.,1,Fritz T.,10,...,NaN,NaN,NaN,NaN,-1,-1.0,NaN,NaN,NaN,NaN
26410,2023-03-31,Masters 1000,Outdoor,Hard,Semifinals,3,Khachanov K.,16,Medvedev D.,5,...,NaN,NaN,NaN,NaN,-2,-3.0,NaN,NaN,NaN,NaN
26411,2023-04-01,Masters 1000,Outdoor,Hard,Semifinals,3,Sinner J.,11,Alcaraz C.,1,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN


In [ ]:

print("Soma dos ultimos 3 resultados de quem ganhou com ranking maior: ", treated_df_winner_bigger_rank['3_matches_sum_y'].fillna(0).mean())

print("Soma dos ultimos 3 resultados de quem ganhou com ranking menor: ", treated_df_winner_loser_rank['3_matches_sum_y'].fillna(0).mean())
      
print("Soma dos ultimos 3 resultados de quem ganhou no total: ", treated_df['3_matches_sum_y'].fillna(0).mean())

Soma dos ultimos 3 resultados de quem ganhou com ranking maior:  0.03871800387180039
Soma dos ultimos 3 resultados de quem ganhou com ranking menor:  0.06654981010809231
Soma dos ultimos 3 resultados de quem ganhou no total:  0.05675235679400295
